In [90]:
#primero importare pandas, numpy y selenium para poder hacer el web scraping
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException
import time
from time import sleep


In [91]:
#seleccionamos la url a la que queremos ir:
url_compe= "https://all.accor.com/ssr/app/ibis/hotels/madrid-spain/open/index.es.shtml?compositions=1&stayplus=false&snu=false&hideWDR=false&accessibleRooms=false&hideHotelDetails=false&dateIn=2025-03-01&nights=1&destination=madrid-spain"

In [92]:
#defino una función que englobe todo el proceso de web scraping y cuyo argumento sea la url
def scrapeo_competencia(url):
    #inicializamos el driver y lo seteamos
    driver = webdriver.Chrome()
    #generamos el driver y abrimos la url, añado 7 segundos de espera
    try:
        driver.get(url)
        time.sleep(7)
    except: 
        print("error al cargar la pagina")
    #obtendre  los hoteles por su clase que es "hotelblock__content"
    try:
        hoteles = driver.find_elements(By.CLASS_NAME,"hotelblock__content")
    except:
        print("error al obtener los hoteles")
    #para cada hotel en la lista de hoteles, obtendre el nombre, valoración y precio del hotel y hago limpieza de los datos en la propia función
    try:
        hoteles_nombre = []
        hotel_valoracion = []
        hotel_precio = []
        for hotel in hoteles:
            nombre = hotel.find_element(By.CLASS_NAME, "title").text.split("\n")[0].replace("á","a").replace("é","e").replace("í","i").replace("ó","o").replace("ú","u")
            valoracion = hotel.find_element(By.CLASS_NAME, "ratings__score").text.split("/")[0]
            precio = hotel.find_element(By.CLASS_NAME, "booking-price").text.replace("€","").replace(" ","")
            hoteles_nombre.append(nombre)
            hotel_valoracion.append(valoracion)
            hotel_precio.append(precio)
    except:
        print("error al obtener los datos de los hoteles")
    #ahora convertiremos las listas en un dataframe
    df=pd.DataFrame({"nombre":hoteles_nombre, "valoracion":hotel_valoracion, "precio":hotel_precio})
    driver.close()
    driver.quit()
    return df.to_csv("hoteles_competencia.csv", index=False)


In [93]:
scrapeo_competencia(url_compe)